## Workspace Management demo

This notebook presents examples of how to manage user workspaces.

In [ ]:
import requests
import datetime

In [ ]:
base_domain = "demo.eoepca.org"
# base_domain = "develop.eoepca.org"
workspaces_endpoint = f'https://workspace-api-open.{base_domain}/workspaces'

We want to associate the workspace with a user. Also, some operations on the workspace require a logged in user. So we are using the client and use Eric as an owner.

In [ ]:
import utils.DemoClient as client
import jwt

demo = client.DemoClient(f"https://test.{base_domain}")
demo.register_client()
demo.save_state()

In [ ]:
USER_ERIC="eric"
USER_ERIC_PASSWORD="defaultPWD"
eric_id_token = demo.get_id_token(USER_ERIC, USER_ERIC_PASSWORD)
headers = {
    'Authorization': 'Bearer ' + eric_id_token
}

jwt_header = jwt.get_unverified_header(eric_id_token)
jwt_payload = jwt.decode(eric_id_token, options={"verify_signature": False})
eric_uid = jwt_payload["sub"]

### Creation and retrieval of credentials

The following request will create a user workspace.

In [ ]:
preferred_workspace_name = f"my-new-user-{datetime.datetime.now().timestamp():.0f}"
preferred_workspace_name

In [ ]:
response = requests.post(
    workspaces_endpoint,
    json={
        "preferred_name": preferred_workspace_name,
        "default_owner": eric_uid,
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
response.json()

Workspace creation is an asynchronous process, so when we try to retrieve the workspace data right now, we see that it is still provisioning.

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()
response.json()

In [ ]:
import time
# wait for provisioning
time.sleep(10)

Now all the credentials should be set up and are available:

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}")
response.raise_for_status()

workspace_data = response.json()
workspace_data["storage"]["credentials"]['secret'] = "hidden_for_demo"
workspace_data["container_registry"]["password"] = "hidden_for_demo"
workspace_data

## Registration

We will now register an ADES instance in the workspace API

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    json={
        "type": "ades",
        "url": "https://demo.pygeoapi.io/stable/processes",
    }
)
response.raise_for_status()
response

Check if the ADES is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?type=application", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

Now we create a new collection:

In [ ]:
collection = requests.get("https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/collection.json").json()

response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register-collection",
    json=collection, # TODO
)
response.raise_for_status()
response

To see if it was registered:

In [ ]:
time.sleep(3)
response = requests.get(f"http://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?title=simple-collection", headers=headers)
response.raise_for_status()
response.json()

## Redeploy

In case something went wrong or an update is necessary, it's possible to trigger the provisioning again:

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/redeploy",
)
response.raise_for_status()
response

## Deletion
Workspaces can also be deleted like this:

In [ ]:
response = requests.delete(
    f"{workspaces_endpoint}/{actual_workspace_name}",
)
response.raise_for_status()
response